### Build a Model, Input to BERT Layer is Embedded (Not Tokens Ids)

In [1]:
import tensorflow as tf
from transformers import AutoTokenizer
from transformers import TFBertForMaskedLM
from transformers import TFCamembertForMaskedLM

In [2]:
### Hyperparameters

vocabSize = 32005
batchSize = 1
sequenceSize = 32
hiddenDimension = 768
outputDimension = 2

In [3]:
# ### Check BERT output

# bertLayer = TFCamembertForMaskedLM.from_pretrained("jplu/tf-camembert-base")
# bertLayer(tensor)

In [4]:
# ### generate a random tensor
# randTensor = tf.random.uniform(shape=[batchSize, sequenceSize, hiddenDimension])

In [5]:
# ### Check BERT output

# bertLayer(None, inputs_embeds=randTensor)

In [6]:
# inp = tf.keras.Input(shape=(32), dtype='int32')
# x = TFCamembertForMaskedLM.from_pretrained("jplu/tf-camembert-base")(inp)[0]
# x = tf.keras.layers.Reshape((sequenceSize*vocabSize,))(x)
# out = tf.keras.layers.Dense(2, activation='softmax')(x)

# model = tf.keras.Model(inp, out)

In [7]:
# inp = tf.keras.Input(shape=(sequenceSize, hiddenDimension), batch_size=batchSize, dtype='float32')
# x = bertLayer(None, inputs_embeds=inp)[0]
# x = tf.keras.layers.Reshape((sequenceSize*vocabSize,))(x)
# out = tf.keras.layers.Dense(2, activation='softmax')(x)

# model = tf.keras.Model(inp, out)

In [8]:
# model(randTensor)

### Build the Classic Model

In [9]:
# ### Build the Classic Model

# inp = tf.keras.Input(shape=(sequenceSize), dtype='int32')
# x = TFCamembertForMaskedLM.from_pretrained("jplu/tf-camembert-base")(inp)[0]
# x = tf.keras.layers.Reshape((sequenceSize*vocabSize,))(x)
# out = tf.keras.layers.Dense(outputDimension)(x)

# modelClassic = tf.keras.Model(inp, out)

In [10]:
# print(len(modelClassic.layers))
# print(len(modelClassic.variables))

# # 4
# # 206

### Build a Model With a Linear Projection at the Beginning

In [11]:
# ### Build a Linear Layer to Embed the tokens_ids

# linearProj = tf.keras.layers.Dense(hiddenDimension, input_shape=(vocabSize,), use_bias=False)

# # # compile layer
# # linearProj.compile()  # dense object does not have attribute compile

# # execute the layer  # executing the layer does not solve the issue
# tensor = tf.zeros(shape=[batchSize, sequenceSize, vocabSize])
# linearProj(tensor)

In [12]:
# ### Build a BERT Layer

# bertLayer = TFCamembertForMaskedLM.from_pretrained("jplu/tf-camembert-base")

# # compile layer
# bertLayer.compile()  # compiling the layer does not solve the issue

# # execute the layer
# randTensor = tf.random.uniform(shape=[batchSize, sequenceSize, hiddenDimension])
# bertLayer(None, inputs_embeds=randTensor)

In [13]:
# ### Build the Experimental Model 0

# # It does not work if batch_size is not specified.

# inp = tf.keras.Input(shape=(sequenceSize, vocabSize), batch_size=batchSize, dtype='float32')
# x = linearProj(inp)
# x = bertLayer(None, inputs_embeds=x)[0]
# x = tf.keras.layers.Reshape((sequenceSize*vocabSize,))(x)
# out = tf.keras.layers.Dense(2, activation='softmax')(x)

# modelExp = tf.keras.Model(inp, out)

In [14]:
# ### Build the Experimental Model 1

# inpA = tf.keras.Input(shape=(sequenceSize), dtype='int32')
# inpB = tf.keras.Input(shape=(sequenceSize, vocabSize), batch_size=batchSize, dtype='float32')
# x = tf.keras.layers.Dense(hiddenDimension, input_shape=(vocabSize,), use_bias=False)(inpB)
# x = TFCamembertForMaskedLM.from_pretrained("jplu/tf-camembert-base")(None, inputs_embeds=x)[0]
# x = tf.keras.layers.Reshape((sequenceSize*vocabSize,))(x)
# out = tf.keras.layers.Dense(outputDimension, activation='softmax')(x)

# modelExp = tf.keras.Model(inputs=[inpA, inpB], outputs=[out])

In [15]:
# # inputs = keras.Input(batch_shape=(batch_size, timesteps, input_dim))

# inpA = tf.keras.Input(shape=(sequenceSize), dtype='int32')
# inpB = tf.keras.Input(batch_shape=(batchSize, sequenceSize, vocabSize), dtype='float32')
# x = tf.keras.layers.Dense(hiddenDimension, input_shape=(vocabSize,), use_bias=False)(inpB)
# x = TFCamembertForMaskedLM.from_pretrained("jplu/tf-camembert-base")(None, inputs_embeds=x)[0]
# x = tf.keras.layers.Reshape((sequenceSize*vocabSize,))(x)
# out = tf.keras.layers.Dense(outputDimension, activation='softmax')(x)

# modelExp = tf.keras.Model(inp, out)

In [16]:
# ### Build the Experimental Model 3

# initialTensor = tf.zeros(shape=[batchSize, sequenceSize, vocabSize])

# inpA = tf.keras.Input(shape=(sequenceSize), dtype='int32')
# inpB = tf.keras.Input(batch_shape=(batchSize, sequenceSize, vocabSize), dtype='float32', tensor=initialTensor)
# x = tf.keras.layers.Dense(hiddenDimension, input_shape=(vocabSize,), use_bias=False)(inpB)
# x = TFCamembertForMaskedLM.from_pretrained("jplu/tf-camembert-base")(None, inputs_embeds=x)[0]
# x = tf.keras.layers.Reshape((sequenceSize*vocabSize,))(x)
# out = tf.keras.layers.Dense(outputDimension, activation='softmax')(x)

# modelExp = tf.keras.Model(inp, out)

In [17]:
# ### Build the Experimental Model 4

# inp = tf.keras.Input(batch_shape=(batchSize, sequenceSize, hiddenDimension), dtype='float32')
# x = TFCamembertForMaskedLM.from_pretrained("jplu/tf-camembert-base")(None, inputs_embeds=inp)[0]
# x = tf.keras.layers.Reshape((sequenceSize*vocabSize,))(x)
# out = tf.keras.layers.Dense(outputDimension, activation='softmax')(x)

# modelExp = tf.keras.Model(inp, out, name='model')

In [18]:
# ### Build the Experimental Model 5

# initialTensor = tf.Variable(initial_value=1., shape=tf.TensorShape(None))

# inp = tf.keras.Input(batch_shape=(batchSize, sequenceSize, hiddenDimension), dtype='float32', tensor=initialTensor)
# x = TFCamembertForMaskedLM.from_pretrained("jplu/tf-camembert-base")(None, inputs_embeds=inp)[0]
# x = tf.keras.layers.Reshape((sequenceSize*vocabSize,))(x)
# out = tf.keras.layers.Dense(outputDimension, activation='softmax')(x)

# modelExp = tf.keras.Model(inp, out, name='model')

In [19]:
# print(len(modelExp.layers))
# print(len(modelExp.variables))

### Build The Experimental Model And Execute It

In [20]:
# ### Build the Experimental Model 0

# inpA = tf.keras.Input(shape=(sequenceSize), dtype='int32')
# x = TFCamembertForMaskedLM.from_pretrained("jplu/tf-camembert-base")(inpA)[0]
# x = tf.keras.layers.Reshape((sequenceSize*vocabSize,))(x)
# out = tf.keras.layers.Dense(outputDimension, activation='softmax')(x)

# modelExp = tf.keras.Model(inpA, out)

In [21]:
### Build the Experimental Model 1

inpA = tf.keras.Input(shape=(sequenceSize), dtype='int32')
inpB = tf.keras.Input(shape=(sequenceSize, hiddenDimension), batch_size=batchSize, dtype='float32')
x = TFCamembertForMaskedLM.from_pretrained("jplu/tf-camembert-base")(inpA, custom_embeds=inpB)[0]
x = tf.keras.layers.Reshape((sequenceSize*vocabSize,))(x)
out = tf.keras.layers.Dense(outputDimension, activation='softmax')(x)

modelExp = tf.keras.Model(inputs=[inpA, inpB], outputs=[out])

In [22]:
print(len(modelExp.layers))
print(len(modelExp.variables))

5
206


In [23]:
### Test the Model on an Input Sentence

tokenizer = AutoTokenizer.from_pretrained("jplu/tf-camembert-base", do_lower_case=True)
sentence = "Elle se situe au cœur d'un vaste bassin sédimentaire aux sols fertiles et au climat tempéré Elle se situe au cœur d'un vaste bassin"
tokens = tokenizer.tokenize(sentence)
tokens_ids = tokenizer.convert_tokens_to_ids(tokens)
sentenceTensor = tf.expand_dims(tf.convert_to_tensor(tokens_ids), 0)
sentenceTensorOneHot = tf.one_hot(sentenceTensor, vocabSize)
randTensor = tf.random.uniform(shape=[batchSize, sequenceSize, hiddenDimension])
print(sentenceTensor.shape)
print(sentenceTensorOneHot.shape)
print(randTensor.shape)

(1, 32)
(1, 32, 32005)
(1, 32, 768)


In [25]:
modelExp(sentenceTensor, randTensor)

KeyError: '139881316355712'